In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import re
import string
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import Counter
from pylab import *
import nltk
import warnings
warnings.filterwarnings('ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


/content/drive/MyDrive/Colab Notebooks/

In [ ]:
data_patio_lawn_garden = pd.read_json('/content/drive/MyDrive/Colab Notebooks/reviews_Patio_Lawn_and_Garden_5.json', lines=True)
data_patio_lawn_garden[['reviewText','overall']].head()

,reviewText,overall
0,Good USA company that stands behind their prod...,4
1,This is a high quality 8 ply hose. I have had ...,5
2,It's probably one of the best hoses I've ever ...,4
3,I probably should have bought something a bit ...,5
4,I bought three of these 5/8-inch Flexogen hose...,5


In [ ]:
#create an instance of WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

Task: We'll  use  a lambda function to extract tokens from each 'reviewText' of this DataFrame, lemmatize them, and concatenate them side by side. We'll use the join function to concatenate a list of words into a single sentence. We'll use the regular expression (re) to replace anything other than letters, digits, and white spaces with blank space.

In [ ]:
nltk.download('wordnet')
nltk.download('punkt')
data_patio_lawn_garden['cleaned_review_text'] = data_patio_lawn_garden['reviewText'].apply(\
lambda x : ' '.join([lemmatizer.lemmatize(word.lower()) \
  for word in word_tokenize(re.sub(r'([^\s\w]|_)+', ' ', str(x)))]))
data_patio_lawn_garden[['cleaned_review_text', 'reviewText', 'overall']].head()

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


,cleaned_review_text,reviewText,overall
0,good usa company that stand behind their produ...,Good USA company that stands behind their prod...,4
1,this is a high quality 8 ply hose i have had g...,This is a high quality 8 ply hose. I have had ...,5
2,it s probably one of the best hose i ve ever h...,It's probably one of the best hoses I've ever ...,4
3,i probably should have bought something a bit ...,I probably should have bought something a bit ...,5
4,i bought three of these 5 8 inch flexogen hose...,I bought three of these 5/8-inch Flexogen hose...,5


Task: Create a DataFrame from tfidf matrix representation of the clead version of review text

In [ ]:
tfidf_model = TfidfVectorizer(max_features=500)
tfidf_df = pd.DataFrame(tfidf_model.fit_transform(data_patio_lawn_garden['cleaned_review_text']).todense())
tfidf_df.columns = sorted(tfidf_model.vocabulary_)
tfidf_df.head()

,10,20,34,8217,able,about,actually,add,after,again,...,work,worked,working,worth,would,yard,year,yet,you,your
0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.120568,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.161561,0.000000
1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
2,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.116566,0.0,0.216988,0.000000,0.049357
3,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.000000
4,0.0,0.0,0.0,0.0,0.0,0.064347,0.0,0.0,0.070857,0.0,...,0.0,0.0,0.0,0.0,0.0,0.083019,0.0,0.000000,0.000000,0.000000


Task:  try to create a new column target which will have 0 if the 'overall' parameter is less than 4, otherwise it will have 1

In [ ]:
data_patio_lawn_garden['target'] = data_patio_lawn_garden['overall'].apply(lambda x : 0 if x<=4 else 1)
data_patio_lawn_garden['target'].value_counts()

1    7037
0    6235
Name: target, dtype: int64

Task: try to use the sklearn's tree() function to fit a decision tree classificaion model on the tf-idf dataframe

In [ ]:
from sklearn import tree
dtc = tree.DecisionTreeClassifier()
dtc = dtc.fit(tfidf_df, data_patio_lawn_garden['target'])
data_patio_lawn_garden['predicted_labels_dtc'] = dtc.predict(tfidf_df)

Task: apply the crosstab function of pandas to compare the results of our classification model with the actual classes of the review

In [ ]:
pd.crosstab(data_patio_lawn_garden['target'], data_patio_lawn_garden['predicted_labels_dtc'])

predicted_labels_dtc,0,1
target,,
0,6227,8
1,1,7036


6627 instances with a target label of 0 correctly classified, and 8 of them wrongly classified. Furthermore, 7036 instances with a target label of 1 are correctly classified, whereas only 1 instance is wrongly classified.

Task: please use sklearn's tree() function to fit a decision tree regression model on the TF-IDF representation of these reviews after cleaning. We'll predict the overall scores using the model.

In [ ]:
from sklearn import tree
dtr = tree.DecisionTreeRegressor()
dtr = dtr.fit(tfidf_df, data_patio_lawn_garden['overall'])
data_patio_lawn_garden['predicted_values_dtr'] = dtr.predict(tfidf_df)
data_patio_lawn_garden[['predicted_values_dtr', 'overall']].head(10)

,predicted_values_dtr,overall
0,4.0,4
1,5.0,5
2,4.0,4
3,5.0,5
4,5.0,5
5,5.0,5
6,5.0,5
7,5.0,5
8,5.0,5
9,4.0,4


from the above result we can see how the actual and predicted score varies for various instances

In [ ]:
# Define a generic function for all classifier model
def clf_model(model_type, X_train, y):
    model = model_type.fit(X_train,y)
    predicted_labels = model.predict(tfidf_df)
    return predicted_labels

1. `def clf_model(model_type, X_train, y):` will define a function named `clf_model` that takes three arguments: `model_type`, `X_train`, and `y`. `model_type` is the machine learning algorithm you want to use (for example, decision tree, random forest, logistic regression, etc.). `X_train` is your training data, and `y` is your training labels.

2. `model = model_type.fit(X_train,y)` This line is fitting the model to the training data. The `fit()` function is a part of every sklearn estimator and is usually the first API method you call after creating an estimator class. This method takes two parameters: the data `X` and the labels `y`. Depending on the estimator, the method might also take other parameters.

3. `predicted_labels = model.predict(tfidf_df)we will predict labels for our test data using the trained model. The `predict()` function is a common method in most sklearn classifiers, it is used for classification, regression, and outlier detection. `tfidf_df` is the test data on which predictions are to be made.

4. `return predicted_labels` The function then returns the predicted labels.

We'll train three kinds of classifier models, namely: random forest, gradient boosting machines, and XGBoost. For random forest, we predict the class labels of the given set of review texts and compare it with their actual class, that is, the target using crosstabs

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=20,max_depth=4,max_features='sqrt',random_state=1)
data_patio_lawn_garden['predicted_labels_rfc'] = clf_model(rfc, tfidf_df, data_patio_lawn_garden['target'])
pd.crosstab(data_patio_lawn_garden['target'], data_patio_lawn_garden['predicted_labels_rfc'])

predicted_labels_rfc,0,1
target,,
0,3302,2933
1,1557,5480


RandomForestClassifier object `rfc` with specified parameters.- `n_estimators=20`: The number of trees in the forest.- `max_depth=4`: The maximum depth of the tree.- `max_features='sqrt'`: The number of features to consider when looking for the best split.- `random_state=1`: Random number seed for reproducibility of results.
.```data_patio_lawn_garden['predicted_labels_rfc'] = clf_model(rfc, tfidf_df, data_patio_lawn_garden['target'])```This line calls some function `clf_model` with parameters `rfc`, `tfidf_df` and `data_patio_lawn_garden['target']`.

`clf_model` is a function that trains the passed classifier (`rfc`) using the passed data (`tfidf_df`) and targets (`data_patio_lawn_garden['target']`). The function probably returns the predicted labels. Those predictions are stored in a new column of `data_patio_lawn_garden` named `'predicted_labels_rfc'`

.```pd.crosstab(data_patio_lawn_garden['target'], data_patio_lawn_garden['predicted_labels_rfc'])```This line uses pandas' `crosstab` function to compute a frequency table of two factors. In this case, it compares the true labels (`data_patio_lawn_garden['target']`) with the predicted labels (`data_patio_lawn_garden['predicted_labels_rfc']`). This table is often used to check the performance of a classification model, with main diagonal of the table (from top left to bottom right) representing correct predictions.

3302 instances with a target label of 2933 correctly classified, and 6134 instances wrongly classified. Furthermore, 5480 instances with a target label of 1 are correctly classified, whereas 1557 instances are wrongly classified

In [ ]:
from xgboost import XGBClassifier
xgb_clf=XGBClassifier(n_estimators=20,learning_rate=0.03,max_depth=5,subsample=0.6,colsample_bytree= 0.6,reg_alpha= 10,seed=42)
data_patio_lawn_garden['predicted_labels_xgbc'] = clf_model(xgb_clf, tfidf_df, data_patio_lawn_garden['target'])
pd.crosstab(data_patio_lawn_garden['target'], data_patio_lawn_garden['predicted_labels_xgbc'])

predicted_labels_xgbc,0,1
target,,
0,3492,2743
1,1414,5623


We will Use the classifier to predict labels for the patio_lawn_garden dataset: `data_patio_lawn_garden['predicted_labels_xgbc'] = clf_model(xgb_clf, tfidf_df, data_patio_lawn_garden['target'])`. The `clf_model` function  trains the provided classifier model (`xgb_clf`) using the provided feature data (`tfidf_df`) and target labels (`data_patio_lawn_garden['target']), and then it uses the trained model to predict labels for the dataset.

In [ ]:
# Task: define a generic function for all regression model
def reg_model(model_type, X_train, y):
    model = model_type.fit(X_train,y)
    predicted_values = model.predict(tfidf_df)
    return predicted_values

Task: try to use the random forest  to predict the overall score of the set of review text.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rfg = RandomForestRegressor(n_estimators=20,max_depth=4,max_features='sqrt',random_state=1)
data_patio_lawn_garden['predicted_values_rfg'] = reg_model(rfg, tfidf_df, data_patio_lawn_garden['overall'])
data_patio_lawn_garden[['overall', 'predicted_values_rfg']].head(10)

,overall,predicted_values_rfg
0,4,4.236717
1,5,4.341767
2,4,4.219413
3,5,4.134852
4,5,4.147218
5,5,4.252751
6,5,4.190971
7,5,4.251688
8,5,4.251610
9,4,4.262498


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
gbr = GradientBoostingRegressor(n_estimators=20,max_depth=4,max_features='sqrt',random_state=1)
data_patio_lawn_garden['predicted_values_gbr'] = reg_model(gbr, tfidf_df, data_patio_lawn_garden['overall'])
data_patio_lawn_garden[['overall', 'predicted_values_gbr']].head(10)

,overall,predicted_values_gbr
0,4,4.354611
1,5,4.441782
2,4,4.329691
3,5,4.060094
4,5,4.145767
5,5,4.162901
6,5,4.227398
7,5,4.146231
8,5,4.269629
9,4,4.136460


In [ ]:
from xgboost import XGBRegressor
xgbr = XGBRegressor(n_estimators=20,learning_rate=0.03,max_depth=5,subsample=0.6,colsample_bytree= 0.6,reg_alpha= 10,seed=42)
data_patio_lawn_garden['predicted_values_xgbr'] = reg_model(xgbr, tfidf_df, data_patio_lawn_garden['overall'])
data_patio_lawn_garden[['overall', 'predicted_values_xgbr']].head(2)

,overall,predicted_values_xgbr
0,4,4.250027
1,5,4.328335
